## Analysis of the matrix, to see if information can be extracted
(glucose & octanoate))

### setup

In [2]:
import extFunc as ext
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]
import numpy as np
from importlib import import_module
import pandas as pd 
import cobra
import optlang
import copy 
import random

In [3]:
%%capture
#Load Models if not already in RAM

#nameList = ['iJN1462_GLC_UR10_9.xml' , 'iJN1462_GLC_UR6_3.xml', 'iJN1462_GLN_UR5_1.xml', 'iJN1462_GLC_UR7_3.xml' , 'iJN1462_OCT_UR3_4.xml']
#nameList = ['iJN1462_GLC_UR6_3.xml' , 'iJN1462_OCT_UR3_4.xml']
nameList = ['iJN1462_OCT_UR3_4.xml','iJN1462_GLC_UR6_3.xml']

if 'modelDict' not in locals():
    modelDict = ext.ImportFunction(nameList)


if 'glc_oct_comp'  not in locals():
    glc_oct_comp = ext.ModelComparison(modelDict=modelDict)


Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


## Matrix extraction
extracting stoch matrix N and dividing it to N_int and N_ext 

In [7]:
#Base case as in Nogales Paper
model_octanoate   = copy.deepcopy(modelDict['iJN1462_OCT_UR3_4'])
model_glucose   = copy.deepcopy(modelDict['iJN1462_GLC_UR6_3'])
#Mat_gluc    =     model_glucose.S
#Mat_oct     =

<bound method Model.internal of <Model M_iJN1463 at 0x7fd2f96b0eb0>>

In [52]:


a = cobra.util.create_stoichiometric_matrix(model_glucose,'dense')

b = cobra.util.array.constraint_matrices(model_glucose, array_type='dense')



In [67]:
print(a.shape)
for i in range(1,a.shape[1]):
    print("")
    #print(len(np.nonzero(a[:,i]))) #.sum()



(2153, 2927)





























































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































shortest path connecting a network buidling a bipartit network usind networkx --> shortest path algorithm